The purpose of this notebook is to calculate distance and time isochrones for a set of points. The points here are the post offices locations for an area which includes Wales and a 20km buffer bordering Wales. In a July 2019 extraction we've got 1202 for this area, out of which 846 only were within Wales' borders.

It is in its current form provided to the reviewers of the paper **"Geographic and Temporal Access to Basic Banking Services Offered through Post Offices in Wales"**. 

The notebook is also a suplementary material to Chapter 4 of the PhD thesis of Andra Sonea.

In order to run this code you need to apply for an API key for https://openrouteservice.org/plans/

In [4]:
#import libraries
import geopandas as gpd
import pandas as pd
import fiona
import matplotlib.pyplot  as plt
from shapely.geometry import shape, Polygon, mapping
from shapely.ops import cascaded_union
import json
from tqdm import tqdm
import time
from openrouteservice import client #THE main library for this script is the one from OSR for calculating isochrones
from descartes import PolygonPatch

In [6]:
po_file = ''


DriverError: ../data/PO_data_Wales20km/PO_data_Wales20km.shp: No such file or directory

In [1]:
# points (post office locations)
po_file = '../data/PO_data_Wales20km/'
po_filename = '../data/PO_2020_Wales20km/' #espg: UK 4326..
pos = gpd.read_file(po_filename)

SyntaxError: starred assignment target must be in a list or tuple (<ipython-input-1-7e3e84118a07>, line 6)

In [3]:
# crs
pos.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
pos.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1364 entries, 0 to 1363
Data columns (total 57 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   code        1364 non-null   object  
 1   link        1364 non-null   object  
 2   processed   1364 non-null   int64   
 3   name        1364 non-null   object  
 4   tel         1364 non-null   object  
 5   street      1364 non-null   object  
 6   city        1364 non-null   object  
 7   postalcode  1364 non-null   object  
 8   country     1364 non-null   object  
 9   latitude    1364 non-null   float64 
 10  longitude   1364 non-null   float64 
 11  MonOpen     1364 non-null   object  
 12  MonClose    1364 non-null   object  
 13  TueOpen     1364 non-null   object  
 14  TueClose    1364 non-null   object  
 15  WedOpen     1364 non-null   object  
 16  WedClose    1364 non-null   object  
 17  ThuOpen     1364 non-null   object  
 18  ThuClose    1364 non-null   object  
 19

In [5]:
if not 'iso-1m' in pos:
    pos['iso-1m'] = ''
if not 'iso-3m' in pos:
    pos['iso-3m'] = ''
if not 'iso-6m' in pos:
    pos['iso-6m'] = ''

In [6]:
#API key for access to ORS to calculate the isochrones
api_key ='5b3ce3597851110001cf62487cfd2206e8ad4ced91cf682b2b778213'
#api_key = '....here is your API key....'
clnt = client.Client(key=api_key)

In [7]:
# Here I split the list of points (post offices) in chunks of 5 becasue the ORS API accepts 5 points/request
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [8]:
#function "get_isochrones" calculates isochrones for the points given. 
#The default selection is for "driving-car" (profile) and "time"(range_type).
def get_isochrones(pointsdf,
                   iso_dict_file,
                   profile='foot-walking', 
                   range_type='time', 
                   range_list=[300, 600, 1200 ], 
                   attributes=['total_pop', 'area']):
    try:
        with open(iso_dict_file, 'r') as f:
            iso_dict = json.load(f)
    except:
        iso_dict = {}
    processed = iso_dict.keys()
    #features = []
    # filter dataframe for items that have already been processed
    num_requests = 0
    sel = pointsdf[~pointsdf['code'].isin(processed)]
    print(f'{len(sel)} rows selected')
    for locations in tqdm(chunks(sel, 5), total=len(sel)/5):
        iso_params = {'locations': [geo.geoms[0].coords[0] for geo in locations['geometry']],
                      'profile': profile,
                      'range_type': range_type,
                      'range': range_list,
                      'attributes': attributes}
        request = clnt.isochrones(**iso_params)
        num_requests += 1
        time.sleep(3)
        #features.extend(request['features'])
        # save results in dataframe
        for index, loc in enumerate(locations.index):
            po_id = pointsdf.at[loc, 'ID']
            iso_1m = request['features'][index*3]
            iso_3m = request['features'][index*3+1]
            iso_6m = request['features'][index*3+2]
            iso_dict[str(po_id)] = {'iso-1m': iso_1m, 'iso-3m': iso_3m, 'iso-6m': iso_6m}
        with open(iso_dict_file, 'w') as f:
            json.dump(iso_dict, f)
    print(f'Executed {num_requests} requests')
    return iso_dict

In [9]:
#execute "get_isochrones" & save all individual time & driving isochrones are saved here in a JSON
iso_dict = get_isochrones(pos, 'Wales_time_isochrones_car_all_test_dec_may22.json')

1364 rows selected


  0%|          | 0/272.8 [00:00<?, ?it/s]


AttributeError: 'Point' object has no attribute 'geoms'

## Process Isochrone Responses

We calculate isochrones also for points in the 20km buffer outside Wales administrative area in order to represent accurately peopple's proximity to a post office even if ocassionally it might be in England. However, in order to calculate the access to a post office only  for the Wales population, we have now to "trim" the isochrones according to the Wales administrative border. We do this below.

In [ ]:
cutter = gpd.read_file('../data/Wales_disolved')

In [ ]:
cutter = cutter.to_crs({'init': 'epsg:4326'})

In [ ]:
cutter.plot()
plt.show()

## Isochrones

In [ ]:
def calc_pop_in_isochrone(isochrone, area_gpd):
    iso_pop = 0
    for lsoa in area_gpd.iterrows():
        intersect = lsoa[1]['geometry'].intersection(isochrone)
        if not intersect.is_empty: 
            iso_pop += lsoa[1]['Popcount']*intersect.area/lsoa[1]['geometry'].area
    return iso_pop

In [ ]:
def save_isochrones(sel_pos, iso_dict, cutter, area, sel_name):
    print('Selecting data...')
    po_ids = [str(po_id) for po_id in sel_pos['ID']]
    print('Extractig isochrones for selected POs...')
    isos_1m = [shape(v['iso-1m']['geometry']) for k, v in iso_dict.items() if k in po_ids]
    isos_3m = [shape(v['iso-3m']['geometry']) for k, v in iso_dict.items() if k in po_ids]
    isos_6m = [shape(v['iso-6m']['geometry']) for k, v in iso_dict.items() if k in po_ids]
    print(f'Selected for 1m iso: {len(isos_1m)}')
    print(f'Selected for 3m iso: {len(isos_3m)}')
    print(f'Selected for 6m iso: {len(isos_6m)}')
    print('Calculating unions...', end='')
    iso_union_1m = cascaded_union(isos_1m)
    print(' 1m ', end='')
    iso_union_3m = cascaded_union(isos_3m)
    print(' 3m ', end='')
    iso_union_6m = cascaded_union(isos_6m)
    print(' 6m')
    print('Clipping polygons...', end='')
    clip_iso_union_1m = iso_union_1m.intersection(cutter)
    print(' 1m ', end='')
    clip_iso_union_3m = iso_union_3m.intersection(cutter)
    print(' 3m ', end='')
    clip_iso_union_6m = iso_union_6m.intersection(cutter)
    print(' 6m')
    print('Save clipped polygons...')
    schema = {'geometry': 'MultiPolygon', 'properties': {'id': 'int', 'desc': 'str'},}

    # threshold 1
    with fiona.open(f'unionisochrones_{sel_name}_t1', 'w', 'ESRI Shapefile', schema) as c:
        ## If there are multiple geometries, put the "for" loop here
        c.write({
            'geometry': mapping(clip_iso_union_1m),
            'properties': {'id': 1, 'desc': f't1_{sel_name}'},
        })
        print(' t1 ', end='')

    # threshold 2
    with fiona.open(f'unionisochrones_{sel_name}_t2', 'w', 'ESRI Shapefile', schema) as c:
    
        c.write({
            'geometry': mapping(clip_iso_union_3m),
            'properties': {'id': 2, 'desc': f't2_{sel_name}'},
        })
        print(' t2 ', end='')

    # threshold 3
    with fiona.open(f'unionisochrones_{sel_name}_t3', 'w', 'ESRI Shapefile', schema) as c:

        c.write({
            'geometry': mapping(clip_iso_union_6m),
            'properties': {'id': 3, 'desc': f't3_{sel_name}'},
        })
        print(' t3 ', end='')
    print('Calculate population...')
    print('t1:', calc_pop_in_isochrone(clip_iso_union_1m, area))
    print('t2:', calc_pop_in_isochrone(clip_iso_union_3m, area))
    print('t3:', calc_pop_in_isochrone(clip_iso_union_6m, area))

## Foot isochrones

In [ ]:
with open('Wales_time_isochrones_foot_all.json', 'r') as f:
    read_iso_dict = json.load(f)

len(read_iso_dict)

read_iso_dict['9526'].keys()

plt.hist(pos['Capacity'], bins=20)
plt.show()

In [ ]:
selected_pos = pos
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'time_foot_all_20200902')

#distance #foot #population #all points 1m: 1834111.3762434889
3m: 2696254.3928601057
6m: 3052516.9274123367

#distance  #car #population #all points 1m: 1655394.2489389551
3m: 2548059.3218654613
6m: 2975395.07296629

#time #car #population #all points
1m: 2462718.3265495286
3m: 2860032.3855188238
6m: 3026030.1623279317

#time #foot #population
1m: 332538.3877926924
3m: 993834.5043317567
6m: 1809514.6763292155

In [ ]:
selected_pos = pos[pos['Capacity'] <= 0.75]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'time_foot_le075_20200902')

#distance #foot #population capacity <0.75 1m: 161511.78725856345
3m: 887967.2199437746
6m: 2253586.4469052646

#distance #car #population capacity <0.75 1m: 126779.94352610075
3m: 784431.8668233516
6m: 2121499.980636307

#time #car #population capcaity <0.75 1m: 488935.52837322565
3m: 1529147.5583683248
6m: 2823674.1243742583

#time #foot #population capcaity <0.75
1m: 13658.711150254983
3m: 45700.529385591995
6m: 151287.4951599431

In [ ]:
selected_pos = pos[pos.eval("Capacity > 0.75")]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'time_foot_gt075_20200902')

## Foot distance

In [ ]:
with open('Wales_distance_isochrones_foot_all.json', 'r') as f:
    read_iso_dict = json.load(f)

len(read_iso_dict)

read_iso_dict['9526'].keys()

plt.hist(pos['Capacity'], bins=20)
plt.show()

In [ ]:
selected_pos = pos
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'distance_foot_all_20200902')

In [ ]:
selected_pos = pos[pos['Capacity'] <= 0.75]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'distance_foot_le075_20200902')

In [ ]:
selected_pos = pos[pos.eval("Capacity > 0.75")]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'distance_foot_gt075_20200902')

## Car Time

In [ ]:
with open('Wales_time_isochrones_car_all.json', 'r') as f:
    read_iso_dict = json.load(f)

len(read_iso_dict)

read_iso_dict['9526'].keys()

plt.hist(pos['Capacity'], bins=20)
plt.show()

In [ ]:
selected_pos = pos
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'time_car_all_20200902')

In [ ]:
selected_pos = pos[pos['Capacity'] <= 0.75]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'time_car_le075_20200902')

In [ ]:
selected_pos = pos[pos.eval("Capacity > 0.75")]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'time_car_gt075_20200902')

## Car Distance

In [ ]:
with open('Wales_distance_isochrones_car_all.json', 'r') as f:
    read_iso_dict = json.load(f)

len(read_iso_dict)

read_iso_dict['9526'].keys()

plt.hist(pos['Capacity'], bins=20)
plt.show()

In [ ]:
selected_pos = pos
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'distance_car_all_20200902')

In [ ]:
selected_pos = pos[pos['Capacity'] <= 0.75]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'distance_car_le075_20200902')

In [ ]:
selected_pos = pos[pos.eval("Capacity > 0.75")]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'distance_car_gt075_20200902')

### End

In [ ]:
selected_pos = pos[pos['Capacity'] > 1.25]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'distance_foot_more125_pop')

#distance #foot #population capacity >1.25 1m: 1196400.1917957608
3m: 2320560.8703956916
6m: 2903603.6611012723

#distance #car #population capacity >1.25 1m: 1048198.9320013712
3m: 2185766.780590868
6m: 2815950.4714616993

#time #car #population capacity >1.25 1m: 1946908.9840998615
3m: 2629013.00317628
6m: 2992139.6564869997

#time # foot #population
1m: 162429.62690606393
3m: 521922.5303261459
6m: 1178162.8030682425

In [ ]:
selected_pos = pos[pos.eval("Capacity >= 0.75 & (Capacity <= 1.25)")]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'distance_foot_more075_less125_pop')

#distance #foot #population Capacity >= 0.75 & (Capacity <= 1.251m: 1242574.5930554967
3m: 2343632.7947459193
6m: 2867983.111124717

#distance #car #population Capacity >= 0.75 & (Capacity <= 1.25
1m: 1048046.2664724686
3m: 2200247.49172126
6m: 2785322.061400475

#time #car #population Capacity >= 0.75 & (Capacity <= 1.25 1m: 1952136.3030312797
3m: 2608445.440415068
6m: 2974161.986355843

#time #foot #population
1m: 158946.40374611082
3m: 513315.9559725913
6m: 1216681.4056649595

In [ ]:
selected_pos = pos[pos.eval("Capacity > 0.5")]
save_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area, 'time_foot_more_05_pop')

#time #foot #population #capacity > 0.5 1m: 320188.2003428298
3m: 962892.6536046048
6m: 1758692.138481088

#time #car #population #capacity > 0.5 1m: 2380834.686981266
3m: 2808104.1988565507
6m: 3018292.4919935074

#distance #car #population #capacity > 0.5 1m: 1600614.592464248
3m: 2474835.135703648
6m: 2944076.482556418

#distance #foot #population #capacity > 0.5 1m: 1777765.6536669862
3m: 2612316.6995851076
6m: 3022578.5720008737

#time #foot #population #capacity > 0.75 1m: 319275.4954136181
3m: 961191.7893956628
6m: 1755328.6549234632

#time #car #population #capacity > 0.75 1m: 2372196.1240560506
3m: 2801337.1425495897
6m: 3018015.4185524266

#distance #car #population #capacity > 0.75 1m: 1596450.2149904328
3m: 2466403.9931703885
6m: 2939527.9850479085

#distance #foot #population #capacity > 0.75
1m: 1773324.282094911
3m: 2603314.070032964
6m: 3017267.672621937

In [ ]:
def plot_isochrones(sel_pos, iso_dict, cutter, area, title):
    print('Selecting data...')
    po_ids = [str(po_id) for po_id in sel_pos['ID']]
    print('Extractig isochrones for selected POs...')
    isos_1m = [shape(v['iso-1m']['geometry']) for k, v in iso_dict.items() if k in po_ids]
    isos_3m = [shape(v['iso-3m']['geometry']) for k, v in iso_dict.items() if k in po_ids]
    isos_6m = [shape(v['iso-6m']['geometry']) for k, v in iso_dict.items() if k in po_ids]
    print(f'Selected for 1m iso: {len(isos_1m)}')
    print(f'Selected for 3m iso: {len(isos_3m)}')
    print(f'Selected for 6m iso: {len(isos_6m)}')
    print('Calculating unions...', end='')
    iso_union_1m = cascaded_union(isos_1m)
    print(' 1m ', end='')
    iso_union_3m = cascaded_union(isos_3m)
    print(' 3m ', end='')
    iso_union_6m = cascaded_union(isos_6m)
    print(' 6m')
    print('Clipping popygons...')
    clip_iso_union_1m = iso_union_1m.intersection(cutter)
    print(' 1m ', end='')
    clip_iso_union_3m = iso_union_3m.intersection(cutter)
    print(' 3m ', end='')
    clip_iso_union_6m = iso_union_6m.intersection(cutter)
    print(' 6m')
    print('Plotting...')
    f, ax = plt.subplots(1, figsize=(12, 12))
    # Extract limits for UK map
    # w, s, e, n = area.total_bounds
    # Center figure on the limits of Wiltsh
    # ax.set_xlim((w, e))
    # ax.set_ylim((s, n))
    area.plot(facecolor='none', edgecolor='gray', linewidth=0.1, ax=ax)
    ax.set_aspect('equal', 'datalim')
    ax.add_patch(PolygonPatch(clip_iso_union_6m, fc='gray', ec='gray', alpha=0.25))
    ax.add_patch(PolygonPatch(clip_iso_union_3m, fc='gray', ec='gray', alpha=0.5))
    ax.add_patch(PolygonPatch(clip_iso_union_1m, fc='gray', ec='gray', alpha=0.75))
    # Add figure title
    f.suptitle(title)
    plt.show()    

### All POs 

In [ ]:
selected_pos = pos

In [ ]:
plot_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area,
                title='Wales\nAccess Isochrones Foot 1m, 3m 6m\nAll Post Offices')

## Capacity >= 23h / week

In [ ]:
selected_pos = pos[pos['Capacity'] >= 0.5]

In [ ]:
plot_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area,
                title='Wales\nAccess Isochrones 1m, 3m 6m\nPost Offices with office hours >= 23h / week')

## Capacity > 46 h / week

In [ ]:
selected_pos = pos[pos['Capacity'] > 1.0]

In [ ]:
plot_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area,
                title='Wales\nAccess Isochrones 1m, 3m 6m\nPost Offices with office hours > 46h / week')

In [ ]:
# <0.75

In [ ]:
selected_pos = pos[pos['Capacity'] < 0.75]

In [ ]:
plot_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area,
                title='Wales\nAccess Time Foot Isochrones 5, 10, 20 mins \nPost Offices with capacity <0.75 ')

In [ ]:
#>1.25

In [ ]:
selected_pos = pos[pos['Capacity'] > 1.25]

In [ ]:
plot_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area,
                title='Wales\nAccess Time Foot Isochrones 5, 10, 20mins\nPost Offices with capacity > 1.25')

In [ ]:
# btw 0.75 and 1.25

In [ ]:
selected_pos = pos[pos.eval("Capacity >= 0.75 & (Capacity <= 1.25)")]

In [ ]:
plot_isochrones(selected_pos, read_iso_dict, cutter.geometry[0], area,
                title='Wales\nAccess Time Foot Isochrones 5, 10, 15mins\nPost Offices with capacity <=0.75 and >= 1.25')

## Population based on Output Area

In [ ]:
# all OA in UK
OA_UK_path = ('../data/infuse_oa_lyr_2011_clipped')
OA_UK = gpd.read_file(OA_UK_path)

In [ ]:
OA_Wales = OA_UK[OA_UK['geo_code'].str.startswith('W')]

In [ ]:
OA_Wales

In [ ]:
OA_Wales_geo_path = '../data/OA_Wales_geo'
OA_Wales.to_file(OA_Wales_geo_path)

In [ ]:
OA_Wales_pop_path = '../data/COA_Wales_population_2019.csv'
OA_Wales_pop = pd.read_csv(OA_Wales_pop_path)

In [ ]:
OA_Wales_pop

In [ ]:
OA_Wales = OA_Wales.merge(OA_Wales_pop, left_on='geo_code', right_on='OA11CD')

In [ ]:
OA_Wales.to_crs('epsg:4326', inplace=True)

In [ ]:
OA_Wales.to_file(OA_Wales_geo_path)

In [ ]:
range_types = ['distance', 'time']
profiles = ['car', 'foot']
capacities = ['all', 'gt075', 'le075']
thresholds = ['t1', 't2', 't3']
date_stamp = '20200902'

In [ ]:
calculations = pd.DataFrame()

for range_type in range_types:
    for profile in profiles:
        for capacity in capacities:
            for threshold in thresholds:
                iso_file = f'unionisochrones_{range_type}_{profile}_{capacity}_{date_stamp}_{threshold}'
                isochrone = gpd.read_file(iso_file)
#                 if isochrone.crs is None:
#                     isochrone.crs = {'init': 'epsg:4326', 'no_defs': True}
                iso_shape = isochrone.iloc[0].geometry
                population = 0
                for index, oa in tqdm(OA_Wales.iterrows(), total=len(OA_Wales)):
                    intersect = oa.geometry.intersection(iso_shape)
                    population += float(oa.Population.replace(',', '')) * intersect.area / oa.geometry.area
                calculations.append({'type': range_type,
                                     'profile': profile,
                                     'capacity': capacity,
                                     'threshold': threshold,
                                     'pop': population}, ignore_index=True)
                print(f'added: {range_type}-{profile}-{capacity}-{threshold} = {population}')